# MMA 662 - Quiz 1

# Before answering any questions, read the following instructions carefully!

1. This quiz has 110 points, so it has ten bonus points, but the grade cannot be more than 100: $\text{grade} = \min\{\text{points earned}, 100\}$
2. Answer the questions in this Jupyter Notebook file and upload it in Quiz 1 entry under the Assignments tab in myCourses. Late submissions are NOT accepted. The deadline for section 075 is 2:25 PM, and section 076's is 5:25 PM. To prevent submission-related issues, please start submitting a few minutes before the deadline. 
3. You can submit your answer file multiple times, but only the last submission is kept and graded. 
4. For each problem, coding or descriptive, please answer them in the cell dedicated to them. We only grade the final answer and not the code. However, we must be able to replicate your results by running your code. Additionally, your code for different problems should run independently. For example, to replicate your results for Question 3, one should not be forced to run your code for Question 2 beforehand. **They must be independent.**
5. the final answer must be printed and visible for coding questions. For descriptive questions, be concise and clear.
6. In questions relevant to BalancedMilk, we have provided the information in its original format - before the shutdown of $S_8$. Make sure to adjust it for each question if necessary.

In [1]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *

# Part 1: Linear Regression (35%): Entree

Linear regression is the cornerstone of modern statistics by which a descriptive analyst fits a straight line to data. Let $(x_i, y_i)$, for $i=1, \cdots n$ be the data points and $a$ and $a_0$ the coefficients of the regression line $\hat{y}=ax+a_0$. Here $x$ is the vector of features (also known as independent variables, covariates, or explanatory variables) and $a$ the vector of coefficients. The coefficient $a_0$ is a constant.

The key question is **how to estimate the coefficients**. A sensible approach is that the best line should minimize the prediction error, $y-\hat{y}$.  This can be done in different ways each with its advantages and limitations which is why to one task in machine learning (such as classification, clustering, etc) there are different algorithms.

The most employed technique is the method of least squares (Ordinary Least Squares or OLS) which you are likely familiar with. Here, we look into two other methods to perform linear regression.

## Data:

We will use two linear regressions to estimate the stock index (as the response variable or dependent variable) of a fictitious economy by using two features:
- Interest Rate
- Unemployment Rate

Therefore, the linear regression model is:
$$\text{Stock Index} = a_1 \text{Interest Rate} + a_2 \text{Unemployment Rate}+a_0.$$

Here is the data set:


| Year | Month | Interest Rate | Unemployment Rate | Stock Index |
|------|-------|---------------|-------------------|-------------|
| 2017 | 12    | 2.75          | 5.3               | 1464        |
| 2017 | 11    | 2.50          | 5.3               | 1394        |
| 2017 | 10    | 2.50          | 5.3               | 1357        |
| 2017 | 9     | 2.50          | 5.3               | 1293        |
| 2017 | 8     | 2.50          | 5.4               | 1256        |
| 2017 | 7     | 2.50          | 5.6               | 1254        |
| 2017 | 6     | 2.50          | 5.5               | 1234        |
| 2017 | 5     | 2.25          | 5.5               | 1195        |
| 2017 | 4     | 2.25          | 5.5               | 1159        |
| 2017 | 3     | 2.25          | 5.6               | 1167        |
| 2017 | 2     | 2.00          | 5.7               | 1130        |
| 2017 | 1     | 2.00          | 5.9               | 1075        |
| 2016 | 12    | 2.00          | 6.0               | 1047        |
| 2016 | 11    | 1.75          | 5.9               | 965         |
| 2016 | 10    | 1.75          | 5.8               | 943         |
| 2016 | 9     | 1.75          | 6.1               | 958         |
| 2016 | 8     | 1.75          | 6.2               | 971         |
| 2016 | 7     | 1.75          | 6.1               | 949         |
| 2016 | 6     | 1.75          | 6.1               | 884         |
| 2016 | 5     | 1.75          | 6.1               | 866         |
| 2016 | 4     | 1.75          | 5.9               | 876         |
| 2016 | 3     | 1.75          | 6.2               | 822         |
| 2016 | 2     | 1.75          | 6.2               | 704         |
| 2016 | 1     | 1.75          | 6.1               | 719         |


## Problem 1.1: Minimum Sum of Absolute Differences

One way of thinking about minimizing the prediction errors is to minimize the sum of absolute errors:
$$\min_{a_0,a_1,a_2} \sum_{i=1}^{n} |y_i-(a_1x^1_i + a_2x^2_i + a_0)|, $$
where $y_i$ is the stock index, $x^1_i$ is the interest rate, and $x^2_i$ is the unemployment rate for date $i$.

As you may know, the absolute value function is not a linear function. Therefore, this problem, in this format, is an unconstrained nonlinear optimization problem. However, many nonlinear optimization problems can be reformulated as linear optimization problem by introducing **auxiliary variables and constraints**.

Ok. So let's introduce a new set of variables $z_i$ for each $i=1,\cdots,n$ and rewrite the problem as:
$$\min \sum_{i=1}^{n} z_i \\ s.t. \quad z_i = |y_i-(a_1x^1_i + a_2x^2_i + a_0)| \quad i = 1, \dots, n$$

Now, the objective function is linear (yay!), but the constraints are not. Let's use the **nature** of the problem in order to linearize the constraints. We replace each nonlinear constraint with two linear constraints $z_i \geq y_i-(a_1x^1_i + a_2x^2_i + a_0)$ and $z_i \geq -(y_i-(a_1x^1_i + a_2x^2_i + a_0))$. Therefore, our problem becomes:
$$\min \sum_{i=1}^{n} z_i 
\\ s.t. \quad z_i \geq y_i-(a_1x^1_i + a_2x^2_i + a_0) \quad i = 1, \dots, n
\\ \quad z_i \geq -(y_i-(a_1x^1_i + a_2x^2_i + a_0)) \quad i = 1, \dots, n$$

Answer the following questions:

### Question 1: (10%)
Why we can replace each nonlinear constraint with these two linear constraints? 

(Hint: Is it possible that in the optimal solution we have $z_i > |y_i-(a_1x^1_i + a_2x^2_i + a_0)|$ instead of $z_i = |y_i-(a_1x^1_i + a_2x^2_i + a_0)|$? Why?)

**Your answer here:**
Because to satifisfy both constraints, the only value z can take is equal to the absolute value of 𝑦𝑖−(𝑎1𝑥1𝑖+𝑎2𝑥2𝑖+𝑎0).It's not possible for z to be larger than the abosulte value because if it does, the objective can always get a better solution at the optimal point by making z taking the aboslute value of 𝑦𝑖−(𝑎1𝑥1𝑖+𝑎2𝑥2𝑖+𝑎0) (since this is a minimization problem). The two constraints will force z to be at least larger than or equal to the absolute value of 𝑦𝑖−(𝑎1𝑥1𝑖+𝑎2𝑥2𝑖+𝑎0), and since the minization problem will force z to be at least smaller or equal to the absolute value, z must eqaul to the absolute value of 𝑦𝑖−(𝑎1𝑥1𝑖+𝑎2𝑥2𝑖+𝑎0).

### Question 2: (10%)

In the following code, the response variable, the independent variables, decision variables, as well as the objective function for this problem are defined. Complete the code by adding the constraints. 

Report the optimal value of the three coefficients and the optimal value of the objective function (the optimal minimum sum of absolute differences).

In [6]:
Stock_Market = {'Year': [2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016],
                'Month': [12, 11,10,9,8,7,6,5,4,3,2,1,12,11,10,9,8,7,6,5,4,3,2,1],
                'Interest_Rate': [2.75,2.5,2.5,2.5,2.5,2.5,2.5,2.25,2.25,2.25,2,2,2,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75],
                'Unemployment_Rate': [5.3,5.3,5.3,5.3,5.4,5.6,5.5,5.5,5.5,5.6,5.7,5.9,6,5.9,5.8,6.1,6.2,6.1,6.1,6.1,5.9,6.2,6.2,6.1],
                'Stock_Index_Price': [1464,1394,1357,1293,1256,1254,1234,1195,1159,1167,1130,1075,1047,965,943,958,971,949,884,866,876,822,704,719]        
                }

df = pd.DataFrame(Stock_Market,columns=['Year','Month','Interest_Rate','Unemployment_Rate','Stock_Index_Price']) 

# the response variable is the stock index price:
stock_index = df['Stock_Index_Price']
# the independent variables are the interest rate and the unemployment rate:
interest_rate = df['Interest_Rate']
unemployment_rate = df['Unemployment_Rate']
# the number of observations:
n = len(stock_index)

#######################
# Optimization Problem:
#######################

# Initialize the model:
model = gb.Model("Question2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Create the decision variables:
a0 = model.addVar(vtype=GRB.CONTINUOUS, name='a0', lb= -np.Infinity) # the intercept
a1 = model.addVar(vtype=GRB.CONTINUOUS, name='a1', lb= -np.Infinity) # the coefficient of the interest rate
a2 = model.addVar(vtype=GRB.CONTINUOUS, name='a2', lb= -np.Infinity) # the coefficient of the unemployment rate
z = model.addVars(n, vtype=GRB.CONTINUOUS, name='z') # the auxiliary variables

#############################################################################################################
# Add the constraints:
model.addConstrs(z[i] >= stock_index[i]-a1*interest_rate[i]-a2*unemployment_rate[i]-a0 for i in range (n))
model.addConstrs(z[i] >= -stock_index[i]+a1*interest_rate[i]+a2*unemployment_rate[i]+a0 for i in range (n))




#############################################################################################################

# Set the objective function:
model.setObjective(quicksum(z[i] for i in range(n)), GRB.MINIMIZE)

# Optimize the model:
model.optimize()

# Print the optimal values of the decision variables:
print('Optimal Solution:')
print('a0 = %g' % a0.x)
print('a1 = %g' % a1.x)
print('a2 = %g' % a2.x)
print('The sum of absolute differences = %g' % model.objVal)




Optimal Solution:
a0 = 1604.8
a1 = 348
a2 = -218
The sum of absolute differences = 1226.6


## Problem 1.2: Minimum of the Maximum Absolute Differences

There is yet another way of thinking about minimizing the prediction error.  Instead of minimizing the sum of absolute errors, one can minimize the absolute value of the largest error. In other words, find the coefficients to:
$$\min_{a_0,a_1,a_2} \max_i |y_i-(a_1x^1_i + a_2x^2_i + a_0)|.$$

Again, this problem is an unconstrained nonlinear optimization problem. So let's see what we can do about it. Let us introduce the auxiliary variable $z$ and rewrite the problem as:
$$\min z \\ s.t. \quad z \geq |y_i-(a_1x^1_i + a_2x^2_i + a_0)| \quad i = 1, \dots, n.$$

Better, but still the constraints are nonlinear. Once again, using the **nature** of the problem, we can linearize the constraints by replacing each nonlinear constraint with two linear constraints $z \geq y_i-(a_1x^1_i + a_2x^2_i + a_0)$ and $z \geq -(y_i-(a_1x^1_i + a_2x^2_i + a_0))$. Therefore, our problem becomes:
$$\min z \\ s.t. \quad z \geq y_i-(a_1x^1_i + a_2x^2_i + a_0) \quad i = 1, \dots, n \\ \quad z \geq -(y_i-(a_1x^1_i + a_2x^2_i + a_0)) \quad i = 1, \dots, n.$$

Answer the following question:

### Question 3: (15%)

Use the code you wrote for the previous problem and adjust it for this problem.

Report the optimal value of the three coefficients and the optimal value of the objective function (the max of absolute difference).

In [14]:
Stock_Market = {'Year': [2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2017,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016],
                'Month': [12, 11,10,9,8,7,6,5,4,3,2,1,12,11,10,9,8,7,6,5,4,3,2,1],
                'Interest_Rate': [2.75,2.5,2.5,2.5,2.5,2.5,2.5,2.25,2.25,2.25,2,2,2,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75],
                'Unemployment_Rate': [5.3,5.3,5.3,5.3,5.4,5.6,5.5,5.5,5.5,5.6,5.7,5.9,6,5.9,5.8,6.1,6.2,6.1,6.1,6.1,5.9,6.2,6.2,6.1],
                'Stock_Index_Price': [1464,1394,1357,1293,1256,1254,1234,1195,1159,1167,1130,1075,1047,965,943,958,971,949,884,866,876,822,704,719]        
                }

df = pd.DataFrame(Stock_Market,columns=['Year','Month','Interest_Rate','Unemployment_Rate','Stock_Index_Price']) 

# the response variable is the stock index price:
stock_index = df['Stock_Index_Price']
# the independent variables are the interest rate and the unemployment rate:
interest_rate = df['Interest_Rate']
unemployment_rate = df['Unemployment_Rate']
# the number of observations:
n = len(stock_index)

#######################
# Optimization Problem:
#######################

# Initialize the model:
model = gb.Model("Question2")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Create the decision variables:
a0 = model.addVar(vtype=GRB.CONTINUOUS, name='a0', lb= -np.Infinity) # the intercept
a1 = model.addVar(vtype=GRB.CONTINUOUS, name='a1', lb= -np.Infinity) # the coefficient of the interest rate
a2 = model.addVar(vtype=GRB.CONTINUOUS, name='a2', lb= -np.Infinity) # the coefficient of the unemployment rate
z = model.addVars(n, vtype=GRB.CONTINUOUS, name='z') # the auxiliary variables
z1 = model.addVar(vtype=GRB.CONTINUOUS, name='z1') 

#############################################################################################################
# Add the constraints:
model.addConstrs(z[i] >= stock_index[i]-a1*interest_rate[i]-a2*unemployment_rate[i]-a0 for i in range (n))
model.addConstrs(z[i] >= -stock_index[i]+a1*interest_rate[i]+a2*unemployment_rate[i]+a0 for i in range (n))
model.addConstr(z1 == max_(z), name="max_contraint")




#############################################################################################################

# Set the objective function:
model.setObjective(z1, sense=GRB.MINIMIZE)

# Optimize the model:
model.optimize()

# Print the optimal values of the decision variables:
print('Optimal Solution:')
print('a0 = %g' % a0.x)
print('a1 = %g' % a1.x)
print('a2 = %g' % a2.x)
print('The max absolute differences = %g' % model.objVal)


Optimal Solution:
a0 = 1095.5
a1 = 384
a2 = -150
The max absolute differences = 133.5


# Part 2: BalancedMilk (55%): Plat Principal

You remember the story of *BalancedMilk* and what happened to their major supplier (if not, you may review Assignment 1). We studied a series of problems that the company faced. But an active and innovative company like *BalancedMilk* is always facing new challenges. In this problem, we will study a new set of problems that the company is facing.

## Problem 2.1: How *Fair* is BalancedMilk?

In Problem 9.1 and 9.2 of Assignment 1, we studied situations in which BalancedMilk wanted to make sure that each demand market receives at least some percentage of its demand. These strategies can be categorized under the umbrella of **fairness**, because they are trying to make sure that no demand market is left behind. In this problem, we will study another aspect of fairness.

The situation is similar to Problem 2 of Assignment 1: $S_8$ is shut down, there is no "not delivering" cost, and BalancedMilk is committed to distribute all the milk it receives from the seven suppliers. Its goal, however, is different this time. Instead of minimizing the total transportation cost, BalancedMilk wants to **minimize the max fairness gap**. Before defining the maximum fairness gap, we need to define two things:
1. The "Demand Fulfillment Rate" (DFR) of a demand market: the total amount of milk delivered to a demand market divided by its demand. 
2. The "Fairness Gap" (FG) between two demand markets: the absolute value of the difference between their DSRs.

For example, suppose in the optimal allocation, BalancedMilk delivers 50 tonnes of milk to demand market 2 ($D_2 = 100$) and 90 tonnes of milk to demand market 3 ($D_3 = 150$). Then:
$$DFR_2 = \frac{50}{100} = 0.5, \quad DFR_3 = \frac{90}{150} = 0.6, \quad FG_{2,3} = |0.5 - 0.6| = 0.1$$

Now, the **maximum fiarness gap** is the maximum of all fairness gaps between all pairs of demand markets. For example, suppose there are only three demand markets, A, B, and C, and $DFR_A = 0.5, DFR_B = 0.6, DFR_C = 0.7$. Then, the maximum fairness gap is $MFG = FG_{A,C} = |0.5 - 0.7| = 0.2$.

Now, answer the following questions:

### Question 4: (15%)

You can use the code you wrote for Problem 2 of Assignment 1 and adjust it for this problem. Report the optimal value of the objective function (the minimum max fairness gap), as well as the DFR of each demand market. If you have absolutely no idea how to introduce constraints in your code to model MFG, you can take a look at "Quiz 1 - Hint" file that is also uploaded, which solves a different, but relevant problem.

If you can solve the problem without any coding, go for it. But if you're going down that road, you have to provide strong, convincing explanations. Otherwise, you'll get zero points - the risk of being bold!

(Also, notice that in the following block we have provided the data in its general format - before the shutdown of $S_8$. Make sure to adjust it for the current problem.)

In [40]:
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
        [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 0]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

##############################
# Your code goes here:


model = gb.Model("BalancedMilk_1")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

Z = model.addVars(n_demand, vtype=GRB.CONTINUOUS, name='z') 

max_gap = model.addVar(vtype = GRB.CONTINUOUS, name = "max_gap")
# Set objective
model.setObjective(max_gap, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])
#3.fairness constraints
for j in Range_demand:
    model.addConstr(Z[i] == sum(X[i, j] for i in Range_supply)/ Demand[j])
    
model.addConstrs(max_gap >= Z[i] - Z[j] for i in range(n_demand) for j in range(n_demand))

# Set objective
model.setObjective(max_gap, GRB.MINIMIZE)

# Optimize
model.optimize()

# Print solution
if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the gap is: %g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")

Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply))) 

Solution status is optimal, and the gap is: 0. 

Supply from S_1 to D_4 : 15.945945945945939
Supply from S_1 to D_6 : 94.05405405405406
Supply from S_2 to D_7 : 80.0
Supply from S_3 to D_4 : 100.0
Supply from S_4 to D_3 : 105.40540540540539
Supply from S_4 to D_5 : 43.24324324324322
Supply from S_4 to D_6 : 42.16216216216219
Supply from S_4 to D_10 : 49.189189189189186
Supply from S_5 to D_6 : 32.432432432432456
Supply from S_5 to D_7 : 67.56756756756755
Supply from S_6 to D_1 : 63.243243243243235
Supply from S_6 to D_2 : 70.27027027027026
Supply from S_6 to D_5 : 83.24324324324326
Supply from S_6 to D_8 : 63.243243243243235
Supply from S_7 to D_4 : 17.56756756756758
Supply from S_7 to D_9 : 112.43243243243242
z[0] : 0.7027027027027026
z[1] : 0.7027027027027026
z[2] : 0.7027027027027026
z[3] : 0.7027027027027026
z[4] : 0.7027027027027026
z[5] : 0.7027027027027026
z[6] : 0.7027027027027026
z[7] : 0.7027027027027026
z[8] : 0.7027027027027026
z[9] : 0.7027027027027026


D_1 is experiencin

## Problem 2.2: BalancedMilk is a Business, not a Charity!

In the previous problem, we assumed BlanacedMilk's goal is to be a hero and distribute the milk as fairly as possible. But, in reality, BalancedMilk is a business, and its goal is to maximize its profit. A more logical goal for BalancedMilk is to **maximize its profit while keeping the maximum fairness gap below a certain threshold**. In this problem, we will model this approach.

The situation is similar to Problem 2.1: $S_8$ is shut down, there is no "not delivering" cost, and BalancedMilk is committed to distribute all the milk it receives from the seven suppliers. In this problem, BalancedMilk wants to distribute the milk to minimize the total transportation cost, while keeping the maximum fairness gap (MFG) below a certain threshold - we call it **the fairness threshold**. The threshold is a parameter that BalancedMilk can set. 


### Question 5: (10%)

Suppose BalancedMilk wants to set the fairness threshold to be equal to 0.25. That is, in the optimal solution, $MFG \leq 0.25$. What is the optimal value of the objective function (the total transportation cost)?

You can use the code you wrote for Question 4 and adjust it for this question - don't forget to change the objective function. (Alternatively, you can write the code from scratch.)



In [22]:
# Your code goes here:

Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
        [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 0]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

##############################
# Your code goes here:


model = gb.Model("BalancedMilk_1")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

Z = model.addVars(n_demand, vtype=GRB.CONTINUOUS, name='z') 

max_gap = model.addVar(vtype = GRB.CONTINUOUS, name = "max_gap")
# Set objective
model.setObjective(max_gap, GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])
#3.fairness constraints
for j in Range_demand:
    model.addConstr(Z[i] == sum(X[i, j] for i in Range_supply)/ Demand[j])
    
model.addConstrs(max_gap >= Z[i] - Z[j] for i in range(n_demand) for j in range(n_demand))

model.addConstr(max_gap <= 0.25)

# Set objective
exp = gb.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjective(exp, GRB.MINIMIZE)

# Optimize
model.optimize()

# Print solution

if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")

Solution = model.getAttr('x', X) 
for j in Range_demand:
  # The demands satisfied from the dummy supplier are not satisfied in reality!
  print(DemandMarket[j], 
        "is experiencing a supply shortage of {} tonnes.".format(Demand[j] - sum(Solution[i, j] for i in Range_supply))) 

Solution status is optimal, and the minimum cost is: $13942.4. 

Supply from S_1 to D_9 : 110.0
Supply from S_2 to D_4 : 33.51351351351353
Supply from S_2 to D_6 : 46.48648648648647
Supply from S_3 to D_7 : 100.0
Supply from S_4 to D_2 : 70.27027027027027
Supply from S_4 to D_6 : 122.16216216216218
Supply from S_4 to D_7 : 47.567567567567565
Supply from S_5 to D_4 : 100.0
Supply from S_6 to D_1 : 41.081081081081074
Supply from S_6 to D_5 : 126.48648648648648
Supply from S_6 to D_8 : 63.24324324324324
Supply from S_6 to D_10 : 49.18918918918919
Supply from S_7 to D_1 : 22.162162162162122
Supply from S_7 to D_3 : 105.4054054054054
Supply from S_7 to D_9 : 2.4324324324324067
z[0] : 0.45270270270270274
z[1] : 0.45270270270270274
z[2] : 0.45270270270270274
z[3] : 0.45270270270270274
z[4] : 0.45270270270270274
z[5] : 0.45270270270270274
z[6] : 0.45270270270270274
z[7] : 0.7027027027027027
z[8] : 0.45270270270270274
z[9] : 0.45270270270270274
max_gap : 0.25


D_1 is experiencing a supply shor

### Question 6: (10%)

If we start with a fairness threshold of 0 (unfairness is not allowed) and move up to 1 (anything goes), what happens to the optimal value of the objective function (the total transportation cost) as the fairness threshold becomes looser and looser? Justify your answer.

**Your answer here:**
The transportation cost should go low as the fairness threshold becomes looser. This is because as the threshold gets lower, the company can choose to distribute less milk to the demand centers that bears a high cost to transport while keeping the fairness in check. In extreme case where threshold = 1, company will essentially choose the transportation plan that optimize its cost, the same solution we have in the homework.  

## Problem 2.3: BalancedMilk has to Set its Priorities Straight!

Until now, we have explored two goals that BalancedMilk can pursue in its distribution strategy: (1) minimizing the total transportation cost, and (2) minimizing the maximum fairness gap. 

Recently, the government has announced that it intends to impose a "sustainability" tax on transportation companies, including BalancedMilk. The tax is going to be proportional to the amount of CO2 emissions of BalancedMilk's trucks. In the following table, the number in front of each supply and demand center represents the supply and demand (in tonnes.) Also, for each supply-demand pair, we have depicted the amount of CO2 emission (in KGs) of transporting one tonne of milk from each supplier to each demand market.

| | $D_1$(90)| $D_2$(100) | $D_3$(150) | $D_4$(190) | $D_5$(180) | $D_6$(240) | $D_7$(210) | $D_8$(90) | $D_9$(160) | $D_{10}$(70) |
|--| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| $S_1$(110) |1.5|4|1.5|5|1|5|6|2|1|1|
| $S_2$(80) |3|10|3|4|2.5|2|1.5|1|2|5|
| $S_3$(100) |3|4|3|5|2|1.5|1|5|3|1|
| $S_4$(240) |10|1|2|5|3|5|3|6|2|3|
| $S_5$(100) |1|1|4|3|5|2|6|2|4|3|
| $S_6$(280) |2|4|3|6|2|7|3|3|6|1|
| $S_7$(130) |1|1|2|3|3|5|4|6|3|10|
| $S_8$(440) |3|3|1|1|1|5|2|3|2|4|

For instance, transporting 100 tonnes of milk from $S_1$ to $D_2$ results in 400 Kgs of CO2 emission.

The situation is similar to Problem 2.1: $S_8$ is shut down, there is no "not delivering" cost, and BalancedMilk is committed to distribute all the milk it receives from the seven suppliers. So, in addition to minimum transportation cost and minimum unfairness, BalancedMilk's third goal is to **minimize the total CO2 emission of transporting milk between supply centers and demand markets.** 

As you have studied in the course, there are two ways to deal with the situations in which we have more than one objective function. In this problem we will model both of them. (For simplicity, we only focus on two objective functions: total transportation cost and total CO2 emission.)

### Question 7: The Hierarchical Approach (10%)

Suppose BalancedMilk's top priority is to minimize the total CO2 emission and its second priority is to minimize the total transportation cost. Model this problem using the hierarchical approach to multi objective optimization. Report the optimal value of each objective function.

(Also, notice that in the following block we have provided the data in its general format - before the shutdown of $S_8$. Make sure to adjust it for the current problem.)



In [35]:
Emmission = [[1.5, 4, 1.5, 5, 1, 5, 6, 2, 1, 1],
             [3, 10, 3, 4, 2.5, 2, 1.5, 1, 2, 5],
             [3, 4, 3, 5, 2, 1.5, 1, 5, 3, 1],
             [10, 1, 2, 5, 3, 5, 3, 6, 2, 3],
             [1, 1, 4, 3, 5, 2, 6, 2, 4, 3],
             [2, 4, 3, 6, 2, 7, 3, 3, 6, 1],
             [1, 1, 2, 3, 3, 5, 4, 6, 3, 10],
             [3, 3, 1, 1, 1, 5, 2, 3, 2, 4]]



Measures = ['cost', 'emission']
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
        [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 0]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

##############################
# Your code goes here:


model = gb.Model("BalancedMilk_1")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total distribution cost:
exp = gb.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjectiveN(exp, index = 0, priority = 0)

exp = gb.quicksum(Emmission[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjectiveN(exp, index = 1, priority = 1)

model.ModelSense = GRB.MINIMIZE

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:
for i in range(2):
    objective = model.getObjective(i).getValue()
    print(f"{Measures[i]}: {objective}")



cost: 13380.0
emission: 1560.0


### Question 8: The Weighted-Sum Approach (10%)

Suppose the imposed sustainability tax is 3.5 USD per KG of CO2 emission. Solve this problem using the weighted-sum approach. Report the optimal value of each objective function. 

(Again, in the following block, we have provided the data in its general format - before the shutdown of $S_8$. Make sure to adjust it for the current problem.)

In [37]:
Emmission = [[1.5, 4, 1.5, 5, 1, 5, 6, 2, 1, 1],
             [3, 10, 3, 4, 2.5, 2, 1.5, 1, 2, 5],
             [3, 4, 3, 5, 2, 1.5, 1, 5, 3, 1],
             [10, 1, 2, 5, 3, 5, 3, 6, 2, 3],
             [1, 1, 4, 3, 5, 2, 6, 2, 4, 3],
             [2, 4, 3, 6, 2, 7, 3, 3, 6, 1],
             [1, 1, 2, 3, 3, 5, 4, 6, 3, 10],
             [3, 3, 1, 1, 1, 5, 2, 3, 2, 4]]

Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
        [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 0]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)


Measures = ['trasnportation cost with no emission penalty', 'emission']


##############################
# Your code goes here:

model = gb.Model("BalancedMilk_1")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

# Objective: Minimizing the total distribution cost:
exp = gb.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjectiveN(exp, weight = 1, index = 0)

exp = gb.quicksum(Emmission[i][j]*X[i, j] for i in Range_supply for j in Range_demand)
model.setObjectiveN(exp, weight = 3.5, index = 1)

model.ModelSense = GRB.MINIMIZE

# Constraints:
# 1. Supply constraints:
# For each supply center, the amount of milk supplied from this center should be equal to its production:
for i in Range_supply:
    model.addConstr(sum(X[i, j] for j in Range_demand) == Supply[i], "Supply_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:
for i in range(2):
    objective = model.getObjective(i).getValue()
    print(f"{Measures[i]}: {objective}")



trasnportation cost with no emission penalty: 11590.0
emission: 1760.0


# Problem 3: If BalancedMilk was Prepared... (20%): Dessert

Let's wind the clock back to the time when everything was fine and dandy - to the time when no supply center is shut down. There are some rumors about a new kettel virus. The virus is highly contagious and can spread through air. Things can get ugly if the virus spreads to BalancedMilk's supply centers.

You are a junior analyst at BalancedMilk who, after taking the awesome MMA 662 course at McGill, understands the value of additional information and its effect on decision making. One of the top managers, also a Desautels alumni, talks to you about their concerns. They are worried about the possibility of the virus spreading to the supply centers, and they want to be prepared. The manager informs you that they predict with 50% $S_8$ will be shut down soon. They want to **minimize their expected total cost.** In the following question, there is a not-delivering cost of 70 dollars per tonne of milk for each demand market that is not fully satisfied.

They have negotiated with the suppliers $S_1$, $S_2$, $S_3$, and $S_4$ who have agreed to increase their supply from what they currently provide under one condition: They want to be compensated for the over-demand by 20 dollars per tonne. That is, if $S_1$ increases its supply from the current 110 tonnes to 120 tonnes, BalancedMilk has to pay $20 \times 10 = 200$ dollars to $S_1$. However, there is a catch! Those suppliers, who have also sensed the danger of an outbreak, want BalancedMilk to make its decision about the additional demand **now, before it becomes clear if $S_8$ is shut down or not.** But the good thing is that regardless of the $S_8$'s status, they will deliver the additional supply to BalancedMilk.

Therefore:
**Total cost in a scenario = transportation cost in that scenario + not-delivering cost in that scenario + cost of over-demand**

### Question 9: (20%)
Concidering the transportation cost, the 70 dollar/tonne not-delivering cost, and the 20 dollar/tonne over-demand cost, what is the optimal expected total cost? How much over-demand should BalancedMilk order from each of those four suppliers? (In this problem assume BalancedMilk is not committed to distribute all the milk it receives from the suppliers.)

In [58]:
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51],
        [34, 17, 10, 21, 9, 33, 14, 26, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [110, 80, 100, 240, 100, 280, 130, 440]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]

Supply1 = [110, 80, 100, 240, 100, 280, 130, 0]

n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

##############################
# Your code goes here:

model = gb.Model("BalancedMilk_1")
model.Params.LogToConsole = 0 # Asking Gurobi not to give us all the details!

# Defining the matrix of decision variables:
X = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])

O = model.addVars(4 , lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply_over" for i in range(4)])


X1 = model.addVars(n_supply , n_demand, lb = 0 , vtype = GRB.CONTINUOUS, name = ["Supply1 from "+SupplyCenter[i]+" to "+DemandMarket[j] for i in Range_supply for j in Range_demand])


# Objective: Minimizing the total distribution cost:
exp = gb.quicksum(Cost[i][j]*X[i, j] for i in Range_supply for j in Range_demand)

supl1=gb.quicksum(X[i, j] for i in Range_supply for j in Range_demand)
dem1=1480

supl2=gb.quicksum(X1[i, j] for i in Range_supply for j in Range_demand)
dem2=1480

exp1 = gb.quicksum(Cost[i][j]*X1[i, j] for i in Range_supply for j in Range_demand)
Over = gb.quicksum(O[i] for i in range(4))

model.setObjective(0.5*(exp+70*(dem1-supl1)+20*Over)+0.5*(exp1+70*(dem2-supl2)+20*Over), GRB.MINIMIZE)

# Constraints:
# 1. Supply constraints:

for i in range(4):
    model.addConstr(O[i]>=0)
    
for i in range(4):
    model.addConstr(sum(X[i, j] for j in Range_demand) <= Supply[i]+O[i], "Supply_"+SupplyCenter[i])

for i in range(4,8):
    model.addConstr(sum(X[i, j] for j in Range_demand) <= Supply[i], "Supply_"+SupplyCenter[i])

for i in range(4):
    model.addConstr(sum(X1[i, j] for j in Range_demand) <= Supply1[i]+O[i], "Supply1_"+SupplyCenter[i])

for i in range(4,8):
    model.addConstr(sum(X1[i, j] for j in Range_demand) <= Supply1[i], "Supply1_"+SupplyCenter[i])

# 2. Demand constraints:
# For each demand market, the amount of milk demanded from this market should not surpass its demand:
for j in Range_demand:
    model.addConstr(sum(X[i, j] for i in Range_supply) <= Demand[j], "Demand_"+DemandMarket[j])

for j in Range_demand:
    model.addConstr(sum(X1[i, j] for i in Range_supply) <= Demand[j], "Demand_1"+DemandMarket[j])

# Optimization
model.optimize()

# Printing the results:

if model.status == GRB.OPTIMAL:
   print('Solution status is optimal, and the minimum cost is: $%g. \n' % model.objVal)
   for v in model.getVars():
       if v.x > 0:
           print(v.varName,':', v.x,)
else:
    print('Optimization was stopped with status %d' % model.status)

print("\n")

print(O)

Solution status is optimal, and the minimum cost is: $25035. 

Supply from S_1 to D_9 : 160.0
Supply from S_2 to D_3 : 20.0
Supply from S_2 to D_4 : 10.0
Supply from S_2 to D_8 : 90.0
Supply from S_3 to D_6 : 240.0
Supply from S_3 to D_7 : 210.0
Supply from S_4 to D_2 : 100.0
Supply from S_5 to D_1 : 90.0
Supply from S_5 to D_4 : 10.0
Supply from S_6 to D_5 : 180.0
Supply from S_6 to D_10 : 70.0
Supply from S_7 to D_3 : 130.0
Supply from S_8 to D_4 : 170.0
Supply_over[0] : 50.0
Supply_over[1] : 40.0
Supply_over[2] : 350.0
Supply1 from S_1 to D_9 : 160.0
Supply1 from S_2 to D_3 : 20.0
Supply1 from S_2 to D_4 : 40.0
Supply1 from S_2 to D_8 : 60.0
Supply1 from S_3 to D_6 : 240.0
Supply1 from S_3 to D_7 : 210.0
Supply1 from S_4 to D_2 : 100.0
Supply1 from S_4 to D_4 : 140.0
Supply1 from S_5 to D_1 : 90.0
Supply1 from S_5 to D_4 : 10.0
Supply1 from S_6 to D_5 : 180.0
Supply1 from S_6 to D_8 : 30.0
Supply1 from S_6 to D_10 : 70.0
Supply1 from S_7 to D_3 : 130.0


{0: <gurobi.Var Supply_over[